<a href="https://www.kaggle.com/code/aleksandrmorozov123/image-manipulations?scriptVersionId=231731350" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Convolutional auto-encoder**

In [19]:
!pip install -q torch_snippets
!pip install torchsummary

import torch
import torch.nn as nn
from torch_snippets import *
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torchsummary import summary
device = 'cuda' if torch.cuda.is_available() else 'cpu'
img_transform = transforms.Compose ([
    transforms.ToTensor (),
    transforms.Normalize ([0.5], [0.5]),
    transforms.Lambda (lambda x: x.to(device))
])

trn_ds = FashionMNIST ('/content/', transform = img_transform, train = True, download = True)
val_ds = FashionMNIST ('/content/', transform = img_transform, train = False, download = True)

batch_size = 128
trn_dl = DataLoader (trn_ds, batch_size = batch_size, shuffle = True)
val_dl = DataLoader (val_ds, batch_size=batch_size, shuffle=False)

ERROR: Cannot uninstall executing 2.0.1, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps executing==2.0.1'.


In [10]:
class ConvAutoEncoder (nn.Module):
    def __init__ (self):
        super ().__init__()
        self.encoder = nn.Sequential (
            nn.Conv2d (1, 32, 3, stride = 3, padding = 1), nn.ReLU(True),
            nn.MaxPool2d (2, stride = 2),
            nn.Conv2d (32, 64, 3, stride = 2, padding = 1), nn.ReLU (True),
            nn.MaxPool2d (2, stride = 1)
        )
        self.decoder = nn.Sequential (
            nn.ConvTranspose2d (64, 32, 3, stride = 2), nn.ReLU (True),
            nn.ConvTranspose2d (32, 16, 5, stride = 3, padding = 1), nn.ReLU(True),
            nn.ConvTranspose2d (16, 1, 2, stride = 2, padding = 1), nn.Tanh ()
        )
    def forward (self, x):
         x = self.encoder (x)
         x = self.decoder (x)
         return x
        
model = ConvAutoEncoder().to(device)

summary (model, torch.zeros (2, 1, 28, 28))
        

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 2, 2]            --
|    └─Conv2d: 2-1                       [-1, 32, 10, 10]          320
|    └─ReLU: 2-2                         [-1, 32, 10, 10]          --
|    └─MaxPool2d: 2-3                    [-1, 32, 5, 5]            --
|    └─Conv2d: 2-4                       [-1, 64, 3, 3]            18,496
|    └─ReLU: 2-5                         [-1, 64, 3, 3]            --
|    └─MaxPool2d: 2-6                    [-1, 64, 2, 2]            --
├─Sequential: 1-2                        [-1, 1, 28, 28]           --
|    └─ConvTranspose2d: 2-7              [-1, 32, 5, 5]            18,464
|    └─ReLU: 2-8                         [-1, 32, 5, 5]            --
|    └─ConvTranspose2d: 2-9              [-1, 16, 15, 15]          12,816
|    └─ReLU: 2-10                        [-1, 16, 15, 15]          --
|    └─ConvTranspose2d: 2-11             [-1, 1, 28, 28]           65
| 

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 2, 2]            --
|    └─Conv2d: 2-1                       [-1, 32, 10, 10]          320
|    └─ReLU: 2-2                         [-1, 32, 10, 10]          --
|    └─MaxPool2d: 2-3                    [-1, 32, 5, 5]            --
|    └─Conv2d: 2-4                       [-1, 64, 3, 3]            18,496
|    └─ReLU: 2-5                         [-1, 64, 3, 3]            --
|    └─MaxPool2d: 2-6                    [-1, 64, 2, 2]            --
├─Sequential: 1-2                        [-1, 1, 28, 28]           --
|    └─ConvTranspose2d: 2-7              [-1, 32, 5, 5]            18,464
|    └─ReLU: 2-8                         [-1, 32, 5, 5]            --
|    └─ConvTranspose2d: 2-9              [-1, 16, 15, 15]          12,816
|    └─ReLU: 2-10                        [-1, 16, 15, 15]          --
|    └─ConvTranspose2d: 2-11             [-1, 1, 28, 28]           65
| 

In [16]:
def train_batch (input, model, criterion, optimizer):
    model.train ()
    optimizer.zero_grad ()
    output = model (input)
    loss = criterion (output, input)
    loss.backward ()
    optimizer.step ()
    return loss

@torch.no_grad ()
def validate_batch (input, model, criterion):
    model.eval ()
    output = model (input)
    loss = criterion (output, input)
    return loss

model = ConvAutoEncoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW (model.parameters(), lr = 0.001, weight_decay = 1e-5)

num_epochs = 5
#log = report (num_epochs)

for epoch in range (num_epochs):
    N = len (trn_dl)
    for ix, (data, _) in enumerate (trn_dl):
        loss = train_batch (data, model, criterion, optimizer)
        #log.record (pos = (epoch + (ix + 1)/N), trn_loss = loss, end = '\r')
    #log.report_avgs (epoch + 1)